In [37]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats

#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
from scipy.misc import derivative

#import testing
import sys
sys.path.append("../")
import selection_tests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
#need to replicate table 3?
school_data = pd.read_stata('aerdata_colombia2.dta')
#keep if checkid==1 & age>=9 & age<=25
school_data = school_data[(school_data['checkid']==1) &
                          (school_data['age']>=9) & (school_data['age']<=25) ]
print(school_data.describe())

            vouch0     sex_name      match_i     match_ic     match_i7  \
count  3542.000000  3542.000000  3542.000000  3542.000000  3542.000000   
mean      0.585263     0.494918     0.354037     0.338509     0.331451   
std       0.492746     0.500045     0.478288     0.473269     0.470801   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       1.000000     0.000000     0.000000     0.000000     0.000000   
75%       1.000000     1.000000     1.000000     1.000000     1.000000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

               age        phone  checkid         math         read  \
count  3542.000000  3542.000000   3542.0  3541.000000  3541.000000   
mean     12.670243     0.886222      1.0    14.713037    16.355835   
std       1.295708     0.317586      0.0    20.436213    22.758410   
min      10.000000     0.000000      1.0     0.000000

In [39]:
# Fit the model with ols = True
x_names =['vouch0', 'age', 'sex_name'] 
y_name1 = 'read'
y_name2 = 'math'

#setup x/y for missing data models
x = school_data[x_names].copy()
y1 = school_data[y_name1]
y2 = school_data[y_name2]

print(stats.mode(school_data[y_name1])[0][0])
print(stats.mode(school_data[y_name2])[0][0])

x = sm.add_constant(x[~np.isnan(y1)])
y1 = y1[~np.isnan(y1)]
y2 = y2[~np.isnan(y2)]

0.0
0.0


/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [40]:
cc_model1 = selection_tests.Tobit(y1,x, cc=True).fit()
cc_model2 = selection_tests.Tobit(y2,x, cc=True).fit()
#why is there randomly a missing observation?
print(cc_model1.summary(title='OLS CC'))
print(cc_model2.summary(title='OLS CC'))

Optimization terminated successfully.
         Current function value: 1.356813
         Iterations: 114
         Function evaluations: 189
Optimization terminated successfully.
         Current function value: 1.315860
         Iterations: 102
         Function evaluations: 172
                                    OLS CC                                    
Dep. Variable:                   read   Log-Likelihood:                -4804.5
Model:                          Tobit   AIC:                             9617.
Method:            Maximum Likelihood   BIC:                             9642.
Date:                Tue, 20 Jul 2021                                         
Time:                        16:52:19                                         
No. Observations:                3541                                         
Df Residuals:                    3537                                         
Df Model:                           3                                         
         

In [41]:
# Fit the model with ols = False
tobit_model1 = selection_tests.Tobit(y1,x).fit()
tobit_model2 = selection_tests.Tobit(y2,x).fit()
print(tobit_model1.summary())
print(tobit_model2.summary())

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


                                Tobit Results                                 
Dep. Variable:                   read   Log-Likelihood:                -7519.5
Model:                          Tobit   AIC:                         1.505e+04
Method:            Maximum Likelihood   BIC:                         1.507e+04
Date:                Tue, 20 Jul 2021                                         
Time:                        16:52:22                                         
No. Observations:                3541                                         
Df Residuals:                    3537                                         
Df Model:                           3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        304.9482     12.413     24.568      0.000     280.620     329.276
vouch0         8.8082      1.995      4.415      0.0

/home/erichschulman/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [53]:
def avg_effect(model,ys,xs):
    params=  model.params
    mux =  np.matmul(np.array(xs),params[0:-1]) 
    res = stats.norm.cdf(mux/np.sqrt(np.abs(params[-1])))*params[1]
    
    #standard errors with delta method...
    cov = model.normalized_cov_params
    g_deriv = []
    for i in range(model.params.shape[0]):
        def g(theta):
            new_params = params.copy()
            new_params[i] = theta
            new_mux =  np.matmul(np.array(xs),new_params[0:-1])
            new_res = stats.norm.cdf(new_mux/np.sqrt(np.abs(new_params[-1])))*new_params[1]
            return new_res.mean()
        
        g_deriv.append(derivative(g,params[i],dx=10e-8))
    g_deriv = np.array([g_deriv])
    var = g_deriv.dot(cov).dot(g_deriv.T)
    return res.mean(), var[0][0]
    
    
    return res.mean(), bse[1]*var.mean()

def marginal_effect(model,ys,xs):
    #start with marginal effects
    params=  model.params
    mux =  np.matmul(np.array(xs.mean(axis=0)),params[0:-1])
    res = stats.norm.cdf(mux/np.sqrt(np.abs(params[-1])))*params[1]
    
    #standard errors with delta method...
    cov = model.normalized_cov_params
    g_deriv = []
    for i in range(model.params.shape[0]):
        def g(theta):
            new_params = params.copy()
            new_params[i] = theta
            new_mux =  np.matmul(np.array(xs.mean(axis=0)),new_params[0:-1])
            return stats.norm.cdf(new_mux/np.sqrt(np.abs(new_params[-1])))*new_params[1]
        
        g_deriv.append(derivative(g,params[i],dx=10e-8))
    g_deriv = np.array([g_deriv])
    var = g_deriv.dot(cov).dot(g_deriv.T)
    return res.mean(), var[0][0]

print(avg_effect(tobit_model1, y1,x))
print('----')
print(marginal_effect(tobit_model1, y1,x))
print('----')

print(avg_effect(tobit_model2, y2,x))
print('----')
print(marginal_effect(tobit_model2, y2,x))
print('----')


----
(2.367238724489131, 0.391475027617278)
----
